# Imports

In [5]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Load and process csv data

In [6]:
data = pd.read_csv('../data/train.csv')

In [7]:
texts = data['text']
labels = data['class']

## Convert texts to TF-IDF vectors

In [8]:
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(texts).toarray()
y = np.array(labels)

## Create a PyTorch dataset

In [9]:
class TextDataset(Dataset):
    def __init__(self, data, targets):
        self.data = torch.tensor(data, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.long)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_dataset = DataLoader(TextDataset(X_train, y_train), batch_size=32, shuffle=True)
test_dataset = DataLoader(TextDataset(X_test, y_test), batch_size=32)

# Basic NN

## Model Class

In [10]:
# Define the FCNN model
class FCNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FCNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

## Initialize Model, Loss and Optimizer

In [11]:
fcnn = FCNN(input_size=1000, hidden_size=256, num_classes=21)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(fcnn.parameters(), lr=0.001)

## Training and Testing

In [12]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for step, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()
        if step % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, step * len(data), len(train_loader.dataset),
                100. * step / len(train_loader), loss.item()))

In [13]:
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += loss_function(output, target).item() * target.size(0)
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), accuracy))

In [14]:
num_epochs = 5
for epoch in range(1, num_epochs + 1):
    train(fcnn, train_dataset, optimizer, epoch)
    test(fcnn, test_dataset)

Train Epoch: 1 [0/173579 (0%)]	Loss: 3.050684
Train Epoch: 1 [3200/173579 (2%)]	Loss: 1.485374
Train Epoch: 1 [6400/173579 (4%)]	Loss: 0.801463
Train Epoch: 1 [9600/173579 (6%)]	Loss: 0.608865
Train Epoch: 1 [12800/173579 (7%)]	Loss: 0.466313
Train Epoch: 1 [16000/173579 (9%)]	Loss: 0.394630
Train Epoch: 1 [19200/173579 (11%)]	Loss: 0.453226
Train Epoch: 1 [22400/173579 (13%)]	Loss: 0.166841
Train Epoch: 1 [25600/173579 (15%)]	Loss: 0.241293
Train Epoch: 1 [28800/173579 (17%)]	Loss: 0.455671
Train Epoch: 1 [32000/173579 (18%)]	Loss: 0.337800
Train Epoch: 1 [35200/173579 (20%)]	Loss: 0.228061
Train Epoch: 1 [38400/173579 (22%)]	Loss: 0.077543
Train Epoch: 1 [41600/173579 (24%)]	Loss: 0.141374
Train Epoch: 1 [44800/173579 (26%)]	Loss: 0.067243
Train Epoch: 1 [48000/173579 (28%)]	Loss: 0.370672
Train Epoch: 1 [51200/173579 (29%)]	Loss: 0.255373
Train Epoch: 1 [54400/173579 (31%)]	Loss: 0.161308
Train Epoch: 1 [57600/173579 (33%)]	Loss: 0.111819
Train Epoch: 1 [60800/173579 (35%)]	Loss: 0.